<a href="https://colab.research.google.com/github/Shaffizy/CartReactAssignment/blob/main/Shaffi's_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the Dependencies**
In this code, I imported the following dependencies: Pandas, Scikit-learn, Tensorflow.
Pandas for to visualize the data in this this programming environment
Scikit-Learn to divide the data to test and train data.
Tensorflow for the training of the date when it is split to train and test.We would use the Dense, Embedding and LSTM.

In [9]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data Collection-kaggle API**
This function is used to load the file "kaggle.json" which contains my kaggle information (name, password) to enable the download of a particular kaggle file called "imdb-dataset-of-50k-movie-reviews".

In [12]:
def Data_Collection():
    kaggle_dictionary = json.load(open("kaggle.json"))
    os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
    os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]
Data_Collection()

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

**Data Extraction**
This function is used to unzip the "imdb-dataset-of-50k-movie-reviews" so it can be visualized by pandas.


In [13]:
def Data_Extraction():
  with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
        zip_ref.extractall()
Data_Extraction()


**Data Processing**
In this function I used pandas to read through the data.Then I replaced the "positive" and "negative" value in the "sentiment" column of the *data* that is the variable of the result of the it's visualization by pandas to integer so the machine can read and understand. After that I defined the *train* and *test* data by spliting them with scikit-learn. Tokenization is now carried out to replace the word with corresponding numbers so the machine can easily work with it. While X(train, test) and Y(train, test) are then initialized with the former being the data of values of "review" and the latter beign the data of values of "sentiment".

In [11]:
def Data_Processing():
    data = pd.read_csv("/content/IMDB Dataset.csv")
    data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    print(train_data.shape)
    print(test_data.shape)
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(train_data["review"])

    X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
    X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)
    print(X_train)
    print(X_test)

    Y_train = train_data["sentiment"]
    Y_test = test_data["sentiment"]
    print(Y_train)
    print(Y_test)

    return X_train, X_test, Y_train, Y_test
Data_Processing()

(40000, 2)
(10000, 2)
[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]
39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64
33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


(array([[1935,    1, 1200, ...,  205,  351, 3856],
        [   3, 1651,  595, ...,   89,  103,    9],
        [   0,    0,    0, ...,    2,  710,   62],
        ...,
        [   0,    0,    0, ..., 1641,    2,  603],
        [   0,    0,    0, ...,  245,  103,  125],
        [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32),
 array([[   0,    0,    0, ...,  995,  719,  155],
        [  12,  162,   59, ...,  380,    7,    7],
        [   0,    0,    0, ...,   50, 1088,   96],
        ...,
        [   0,    0,    0, ...,  125,  200, 3241],
        [   0,    0,    0, ..., 1066,    1, 2305],
        [   0,    0,    0, ...,    1,  332,   27]], dtype=int32),
 39087    0
 30893    0
 45278    1
 16398    0
 13653    0
         ..
 11284    1
 44732    1
 38158    0
 860      1
 15795    1
 Name: sentiment, Length: 40000, dtype: int64,
 33553    1
 9427     1
 199      0
 12447    1
 39489    0
         ..
 28567    0
 25079    1
 18707    1
 15200    0
 5857     1
 Name: sentiment, Len

**LSTM - Long Short-Term Memory**
I used this function to build the model with Embedding, LSTM, Dense provided by tensorflow and then I compile the model together so everything works simultaneously in concurrent order and I train the model the finally carry out model evaluation which shows the accuracy of the machine  

In [8]:
def Trainning():
    X_train, X_test, Y_train, Y_test = Data_Processing()
    # build the model

    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation="sigmoid"))
    model.summary()

    # compile the model
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    # training the Model
    model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

    # model Evaluation
    loss, accuracy = model.evaluate(X_test, Y_test)
    print(f"Test Loss: {loss}")
    print(f"Test Accuracy: {accuracy}")

    return model
Trainning()

(40000, 2)
(10000, 2)
[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]
39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64
33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 308s 611ms/step - accuracy: 0.7402 - loss: 0.5140 - val_accuracy: 0.8425 - val_loss: 0.3679
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 322s 611ms/step - accuracy: 0.8492 - loss: 0.3624 - val_accuracy: 0.8640 - val_loss: 0.3350
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 299s 564ms/step - accuracy: 0.8584 - loss: 0.3347 - val_accuracy: 0.8484 - val_loss: 0.3674
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 324s 568ms/step - accuracy: 0.8625 - loss: 0.3242 - val_accuracy: 0.8545 - val_loss: 0.3555
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 326s 577ms/step - accuracy: 0.8807 - loss: 0.2984 - val_accuracy: 0.8750 - val_loss: 0.3122
313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 134ms/step - accuracy: 0.8777 - loss: 0.2976
Test Loss: 0.2972109317779541
Test Accuracy: 0.8809000253677368


**Building a Predictive System**

In [15]:
def predict_sentiment(review):
  Data_Processing()
  model =Trainning()

  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

(40000, 2)
(10000, 2)
[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]
39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64
33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64
(40000, 2)
(10000, 2)
[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
 80/500 ━━━━━━━━━━━━━━━━━━━━ 3:38 520ms/step - accuracy: 0.5820 - loss: 0.6693